In [132]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

confianza = 1.65

datos = pd.read_csv('data/ventas_finales_rellenadas.csv', encoding = "ISO-8859-1")
datos = datos.drop(columns='Unnamed: 0')
datos = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'PERIODO']).sum().reset_index()

datos['estacion_deseada'] = datos.apply(lambda x: x['PERIODO'].split('-')[1] in ('02'), axis=1)

datos = datos[datos['estacion_deseada'] == True]
datos = datos.drop(columns='estacion_deseada')

conversion = pd.read_csv('data/conversiones_unidades.csv', encoding = "ISO-8859-1")[['ID_SKU_VENTA', 'CAJAS_POR_PALLET']]
sku_pallets = {}
for index, row in conversion.iterrows():
    sku_pallets[row['ID_SKU_VENTA']] = row['CAJAS_POR_PALLET']
    
datos['SUM(PREVENTA)'] = datos.apply(lambda x: x['SUM(PREVENTA)'] / sku_pallets[x['ID_SKU_VENTA']], axis=1)
data_agrupada = datos.copy()

In [124]:
def stats(datos):

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'SUM(PREVENTA)': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'SUM(PREVENTA)': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA / (30 / 7), axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD / (30 / 7), axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)

    data_completa['SS'] = data_completa.apply(lambda x: confianza * x['STD'], axis=1)

    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)

    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)

    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

In [134]:
data_no_agrupada_stats, porcentaje_ss_no_agrupado = stats(datos)

In [135]:
"""FILE_AGRUPACIONES = 'data/sku_unification_v6.xlsx'
sheets_dict = pd.read_excel(FILE_AGRUPACIONES, sheet_name=None)

master = pd.DataFrame()
for name in sheets_dict:
    
    df = sheets_dict[name]
    master = pd.concat([master, df])
    
dict_sku_cd = {}
master = master.rename(columns={'SKU': 'ID_SKU_VENTA', 'CD': 'DESCR_CENDIST'})"""
dict_sku_cd = {}
#for index, row in master.iterrows():
for index, row in data_no_agrupada.iterrows():
    
    #if row['CANTIDAD HUB 3R'] != 0:
    if row['COEF_VAR'] >= 0:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1
        
    else:
        dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0
        
hubs_dict = {'ARICA': 'IQUIQUE',
 'IQUIQUE': 'IQUIQUE',
 'ANTOFAGASTA': 'ANTOFAGASTA',
 'COPIAPO': 'COPIAPO',
 'COQUIMBO': 'COQUIMBO',
 'OVALLE': 'COQUIMBO',
 'ILLAPEL': 'CURAUMA',
 'LLAY LLAY': 'CURAUMA',
 'CURAUMA': 'CURAUMA',
 'CERVECERA': 'SANTIAGO SUR',
 'MODELO': 'SANTIAGO SUR',
 'SANTIAGO SUR': 'SANTIAGO SUR',
 'RANCAGUA': 'SANTIAGO SUR',
 'TALCA': 'TALCA',
 'CHILLAN': 'TALCAHUANO',
 'TALCAHUANO': 'TALCAHUANO',
 'LOS ANGELES': 'TALCAHUANO',
 'TEMUCO': 'TEMUCO',
 'VALDIVIA': 'VALDIVIA',
 'OSORNO': 'VALDIVIA',
 'PUERTO MONTT': 'PUERTO MONTT',
 'CASTRO': 'PUERTO MONTT',
 'COYHAIQUE': 'COYHAIQUE',
 'CALAMA': 'ANTOFAGASTA'}
        
data_agrupada['DESCR_CENDIST'] = data_agrupada.apply(lambda x: x['DESCR_CENDIST'] if (x['ID_SKU_VENTA'], x['DESCR_CENDIST']) in dict_sku_cd and dict_sku_cd[(x['ID_SKU_VENTA'], x['DESCR_CENDIST'])] == 0 else hubs_dict[x['DESCR_CENDIST']], axis=1)
data_agrupada = data_agrupada.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'PERIODO']).sum().reset_index()

In [136]:
data_agrupada_stats, porcentaje_ss_agrupado = stats(data_agrupada)

In [137]:
data_no_agrupada_stats

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.884722,0.121571,0.137411,0.200591,1.085314,0.184823
1,515,ARICA,0.105000,0.020207,0.192450,0.033342,0.138342,0.241011
2,515,CALAMA,0.668889,0.107614,0.160885,0.177563,0.846452,0.209773
3,515,CASTRO,0.589167,0.197042,0.334442,0.325120,0.914287,0.355600
4,515,CERVECERA,18.242778,1.634628,0.089604,2.697136,20.939914,0.128804
...,...,...,...,...,...,...,...,...
30669,875364,CURAUMA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30670,875364,SANTIAGO SUR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30671,875365,CURAUMA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
30672,875365,SANTIAGO SUR,0.077778,0.134715,1.732051,0.222280,0.300058,0.740791


In [138]:
data_agrupada_stats

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,1.553611,0.224590,0.144560,0.370573,1.924184,0.192587
1,515,COPIAPO,0.561944,0.110423,0.196502,0.182198,0.744143,0.244843
2,515,COQUIMBO,3.111111,0.963654,0.309746,1.590029,4.701141,0.338222
3,515,COYHAIQUE,0.194444,0.052608,0.270555,0.086803,0.281248,0.308636
4,515,CURAUMA,0.478333,0.197042,0.411935,0.325120,0.803453,0.404653
...,...,...,...,...,...,...,...,...
16064,875364,CURAUMA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16065,875364,SANTIAGO SUR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16066,875365,CURAUMA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16067,875365,SANTIAGO SUR,0.077778,0.134715,1.732051,0.222280,0.300058,0.740791


In [139]:
porcentaje_ss_no_agrupado

0.401840328248477

In [140]:
porcentaje_ss_agrupado

0.3807996088167468